## Asignacion 2

Todos los asignaciones serán presentadas utilizando los cuadernos de [Jupyter Notebook](http://jupyter.org/), además de respectivas pruebas como ejemplo, así como el uso de mediciones de velocidad de ejecución utilizando el comando `timeit` y algunos gráficos si fuese necesario.

### Preguntas

1. Presenta un reporte del siguiente artículo [Data Science, Machine Learning, and Statistics: what is in a name?](http://www.win-vector.com/blog/2013/04/data-science-machine-learning-and-statistics-what-is-in-a-name/).

2. Revisa el simpático proyecto kickstarter [The Emoji Translation Project :The world needs an emoji translation engine. Help us build it using the power of crowdsourcing. ](https://www.kickstarter.com/projects/fred/the-emoji-translation-project). 

3. Leer el artículo de Paul Graham [A Plan for Spam](http://www.paulgraham.com/spam.html) y responder las siguientes  preguntas de acuerdo a lo que dice el autor y lo que piensas tu:

   - ¿Cuáles son los componentes clave de su sistema de filtrado estadístico? En otras palabras, ¿cómo funciona?.
   - Presenta una lista de beneficios del enfoque estadístico.

4. Presenta un reporte de tus ideas con ejemplos  del siguiente artículo [Choosing a Machine Learning Classifier](http://blog.echen.me/2011/04/27/choosing-a-machine-learning-classifier/).

### 1. Data Science, Machine Learning, and Statistics: what is in a name?

En el artículo se menciona que en muchos artículos en internet se les pone el nombre de data science cuando muchas veces no es adecuado usar el término. El autor explica que el termino data science se debe utilizar cuando se enfatiza en las técnicas usadas en el modelamiento de la data y cuando esta es el corazón de la aplicación.

** Estadística **

La estadística también trata con datos, pero con cálculos sobre esos datos que pueden servir para anticipar eventos. El autor menciona que es un campo pesimista ya que si normalmente los experimientos se llevaran a cabo bien en la realidad no se necesitaría de la estadística, por ello la mejor fuente para un trabajo estadístico son los experimentos malos. Por último, la estadística es buena tratando con problemas que tienen data pequeña. 

** Machine Learning **

Machine Learning muchas veces entra en lugares donde la estadística no puede entrar, su objetivo es encontrar modelos que no se diferencien mucho de los correctos a deferencia de la estadística que busca modelos precisos.
En los problemas de predicción machine learning usa la optimización. Una buena práctica en machine learning es usar técnicas de optimización que sean generalizables. 

**Data Mining**

Los fines del data mining son distintos aunque se prefiera usar a veces el témino data science. El objetivo del Data Mining es encontrar relaciones entre la data(clustering y caracterización), mas no hacer predicciones.

**Informática** 

Básicamente tiene que ver con la infraestructura de la data y el uso de algoritmos.

**Big Data ** 

Se usa para hacer análisis predictivo , visualización, reporte y resumen de la data, mas que hacer modelamiento.

** Data Science ** 

Se basa en técnicas y teorías de muchos campos como la matemática, estadística, ingeniería de datos, reconocimiento de patrones y aprendizaje, visualización, etc. Su fin es darle un significado a la data . 

### 2. The Emoji Translation Project

En el proyecto Emoji, Fred Benenson y Chris Mulligan proponen crear un motor de traducción(como google translate) para Emojis.

![alt text](https://tctechcrunch2011.files.wordpress.com/2017/08/1-shpv5hx-voeaxvpfiokaaw.png?w=738 "Android Emojis")
<center>Emojis para Android. Obtenido de [Techcrunch](https://techcrunch.com/2017/08/21/meet-android-oreos-all-new-emoji/)</center>

El problema con la traducción a emoji es que no existe una única y adecuada forma de traducir una frase a emoji, así como una traducción directa de diccionario produce errores por no no respetar la gramática y el contexto. Esto produjo que los intentos antecesores a este proyecto no fueran tan efectivos, como comentan en su video promocional.

Para poder recoger esa variabilidad, requieren de una masiva cantidad (Big Data) de texto existente en ambos lenguajes, a lo que se denomina texto paralelo. El problema es que no existe tanto contenido en emoji a diferencia del inglés (su lengua madre), por lo que requieren traductores y fondos para pagarles.

Con la data mencionada, plantean generar un algoritmo aplicando estadística y machine learning para poder modelar como las ideas son representadas en emojis y como a base de ideas simples se comunican ideas complejas.

Si bien no se detalla en su página de kickstarter, se puede deducir que el algoritmo a usar será supervisado (tiene data comparativa) y de clasificación. Podría usarse frecuencias de frases/palabras y asignar probabilidades de ocurrencia para todas las posibles frases por cada oración, pero resultaría poco escalable. Otra manera es a través del Deep Learning, con redes neuronales recurrentes, como se detalla [aqui](https://medium.com/@ageitgey/machine-learning-is-fun-part-5-language-translation-with-deep-learning-and-the-magic-of-sequences-2ace0acca0aa). 


El motor quedará disponible al público, al igual que la data usada. Así mismo, prometen entregar un libro de frases en emoji a quienes apoyen su causa.

### 3. A plan for Spam
Para explicar un poco el funcionamiento empezaremos con dos conjuntos de correos, uno con spam y el otro que no son spam;cada uno con  4000 mensajes. 

Luego se escanea todo el texto, junto con los encabezados, el html,etc; de cada mensaje. Se ignoran todos los comentarios hmtl y los tokens.

Se cuenta el número de tokens de cada vez ocurre, en ambos conjuntos. Se elabora una tabla hash para realizar esto(para cada conjunto); y luego una tercera tabla hash donde se le asigna una probabilidad(de que un correo que lo contenga sea spam) a cada token.

Para evitar los falsos positivos(correos inocentes que suelen ser confundidos con spam pero que en realidad no lo son) lo que se hizo es duplicar los numeros en la tabla BIEN(tabla del conjunto donde no hay spam). Esto ayuda a distinguir entre las palabras que ocasionalmente ocurren en el correo legítimo y las palabras que casi nunca lo hacen. Luego está la cuestión de qué probabilidad asignar a las palabras que ocurren en un corpus(conjunto) pero no en el otro. Por prueba y error eligen entre 0.01 y 0.99.

Cuando llega un nuevo correo, se escanea en tokens, y los quince tokens más interesantes, donde lo interesante se mide en qué medida su probabilidad de spam es de un 0.5 neutral, se utilizan para calcular la probabilidad de que el correo sea correo no deseado.   

Cuando encontramos un palabra que no se ha visto antes(poco usual) se le asigna 0.4(por prueba y error) ya que las palabras de spam tienden a ser demasiado familiares.

Se trata al correo como correo no deseado si el algoritmo anterior le da una probabilidad de más de 0.9 de ser correo no deseado. Pero en la práctica no importaría mucho donde pongo este umbral, porque pocas probabilidades terminan en el medio del rango.

VENTAJAS:
- No es necesario leer una gran cantidad de correos de los últimos meses para encontrar patrones y redefinirlos, con este enfoque las probabilidades son asignadas a las palabras y las evalua en conjunto.

- Otra ventaja del el enfoque bayesiano es que considera todas las palabras(y la estructura) en el correo electrónico, tanto buenas como malas, y combina las probabilidades de todas ellas.

- Un filtro bayesiano puede ser entrenado en base a cada usuario, es decir, se basa en la actividad de cada usuario, asi como sus preferencia(designa ciertos correos como spam), por lo que las probabilidades de las palabras son únicas para cada usuario.


### 4. Choosing a Machine Learning Classifier

Para empezar, el presente articulo es un post de referencia a que algoritmo de Machine Learning escoger, donde también se muestran las diferencias de algunos de ellos con los otros, tanto en que tipo de datos son eficientes y cuando no. Además, algunas notas del autor son, si el conjunto de entrenamiento es pequeño, los clasificadores de alto sesgo/baja varianza (Naive Bayes) tienen una ventaja sobre los clasificadores de baja polarización/alta varianza (kNN), ya que el último se sobreajustará. Pero los clasificadores de baja polarización/alta varianza comienzan a ganar a medida que su conjunto de entrenamiento crece (tienen un menor error asintótico), ya que los clasificadores de alto sesgo no son lo suficientemente potentes como para proporcionar modelos precisos. A partir de esto, los algoritmos a debatir son:

- **Naive Bayes**: es un clasificador probabilístico fundamentado en el teorema de Bayes y algunas hipótesis simplificadoras adicionales. La ventaja de este algoritmo es que converge más rápido que los modelos discriminatorios de Regresión Logística, pués necesita menor data para capacitarse, una desventaja es que no puede puede aprender las interacciones entre características de los datos, es decir no diferencia relaciones internas entre las caracteristicas de los datos, pues lo considera como un caso implícito. Por ejemplo: entrenar un modelo con datos de altura, peso y talla de calzado, entonces, con estos datos se puede obtener una media y una varianza de estos datos, agrupados por sexo, luego, con el fin de obtener el sexo de cierta persona con cierta talla, peso y calzado, se usa este modelo entrenado, que nos devolvera una probabilad de que sea hombre o mujer.

- **Logistic Regression**: es un tipo de regresión utilizado para predecir el resultado de una variable categórica. Las ventajas de este algoritmo, es cuando se agrega data al modelo, este se puede actualizar usando un método de descenso de gradiente en línea. También se adecua fácilmente a medidas probabilísticas e intervalos de confianza. Por ejemplo, un modelo con data de probabilidad de aprobar vs horas de estudio, es decir, se tiene una tabla con datos de horas de estudio y si aprobo (1) y si jalo (0), luego con el algoritmo de Regresión Logística, se puede determinar una función que ayude a definir que tan probable es aprobar un curso a partir de las horas de estudio.

- **Decision Trees**: es un modelo de predicción basado en árboles, Estos árboles son muy útiles para visualizar las diversas opciones que se tienen y como llegar a ellas, además se les pueden aplicar métodos de inducción, como por ejemplo la inducción hacia atrás. Ventajas de este tipo son, manejan fácilmente las características de cada uno de los datos, no son paramétricas, es decir no importan si los datos son atipicos o si los datos son linealmente separables. Las desventajas son, no son de aprendizaje en linea, es decir, al añadir más data de entrenamiento, este modelo genera un árbol diferente al anterior, para evitar esto, se usan arboles reforzados o bosques aleatorios, los cuales son rápidos y escalables. Un ejemplo de árboles de decision, una tabla de todos los movimientos seguidos por dos jugadores del juego michi. 

- **SVMs**: Dado un conjunto de ejemplos de entrenamiento se puede etiquetar las clases y entrenar una SVM para construir un modelo que prediga la clase de una nueva muestra, es decir, una SVM construye un(os) hiperplano(s) en un espacio de dimensionalidad muy alta (o incluso infinita) que puede ser utilizado en problemas de clasificación o regresión. Además, es popular en problemas de clasificación de texto donde los espacios muy altos son la norma. Por ejemplo, del mismo ejemplo de Naive Bayes, con los mismos datos, se podría clasificar cada uno de ellos separandolos por un hiperplano, que diferenciaría a ambos sexos.

## Lista de ejercicios

Las referencias a los ejercicios son :

   * Asignaciones de Andrew Ng de Stanford University y cursos en Coursera.
   * David Barber, [Bayesian Reasoning and Machine Learning](http://web4.cs.ucl.ac.uk/staff/D.Barber/pmwiki/pmwiki.php?n=Brml.Online), Cambridge University Press, 2017.
   * Pattern Recognition and Machine Learning de Chris Bishop 2006.
   * Machine Learning Refined, Watt, Borhani, Katsaggelos 2016.


**1** . Este conjunto de problemas implica la implementación de varias variantes del algoritmo Perceptron. Antes de poder construir estos modelos y medir su rendimiento, divide tus datos de entrenamiento (es decir, `entrenamiento_spam.txt`) en un conjunto de entrenamiento y validación, poniendo los últimos 1000 correos electrónicos en el conjunto de validación. Por lo tanto, tendrá un nuevo conjunto de entrenamiento con 4000 correos electrónicos y un conjunto de validación con 1000 correos electrónicos. 

Explica por qué medir el rendimiento de su clasificador final sería problemático si no hubiera creado este conjunto de validación.

**Respuesta**

Sería problemático porque puede producir distorsión en los resultados.
El rendimiento del modelo sobre la data de entrenamiento no es un buen indicador del rendimiento predictivo, ya que el modelo puede haber memorizado la data de entrenamiento resultando en un accuracy cercano al 100%; pero para nuevos datos que no se encuentren en la data de entrenamiento, puede que no haga una predección adecuada. A esto se le conoce como overfitting.

<img src="https://chunml.github.io/images/tutorials/underfit-overfit/poly_9_overfit.jpg" width="500">
<center>Ejemplo de Overfitting obtenido del [Tutorial de Trung Tran](https://chunml.github.io/ChunML.github.io/tutorial/Underfit-Overfit/)</center>

Una técnica para evitar esto consiste en, si se tiene una gran cantidad de data, separar una parte para entrenar el modelo (conjunto de entrenamiento) y otra data para evaluarlo (conjunto de validación), permitiendonos verificar que el modelo generalice (aprenda) correctamente.



In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('entrenamiento_spam.txt')#,sep="\s",header=None)

In [2]:
from IPython.core.display import display
data.columns = ['Text']
display(data.head())

# Separamos la columna de Spam
df = pd.DataFrame(data.Text.str.split(' ',1).tolist(),columns=['Spam','Text'])
display(df.head())

,Text
0,1 have tax problem do you ow the ir monei if y...
1,0 r robert harlei write r scuse me for post in...
2,0 on wed number oct number brian fahrland wrot...
3,0 quot ronan waid sure but soft link would do ...
4,0 i notic a low count of razor d spam messag s...


,Spam,Text
0,1,have tax problem do you ow the ir monei if you...
1,0,r robert harlei write r scuse me for post in g...
2,0,on wed number oct number brian fahrland wrote ...
3,0,quot ronan waid sure but soft link would do th...
4,0,i notic a low count of razor d spam messag so ...


In [3]:
X = df['Text']
X = X.to_frame() # Volvemos X nuevamente un DataFrame
y = df['Spam']
test_size = 1.0/5.0 # 1000 correos para validación de los 5000 totales

from sklearn.model_selection import train_test_split
# Partimos la data en un set de entrenamiento y un set de validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size,random_state=2)

print (len(X_train))
X_train.head()

3999


,Text
1305,todai s headlin from the regist to unsubscrib ...
2525,here s clarif of why i did first test result u...
3966,newslett coachnvest septembr number coachnvest...
3394,onc upon a time vill wrote i assum that you wo...
2961,ar your tire of number to number let us show y...


**2** . Transforme todos los datos en vectores de características. Construya una lista de vocabulario usando solamente el conjunto de entrenamiento de 4000 correos electrónicos encontrando todas las palabras que aparecen en el conjunto de entrenamiento.

Ten en cuenta que suponemos que los datos de los conjuntos de validación y prueba no se ven completamente cuando entrenamos nuestro modelo y, por lo tanto, no utilizamos ninguna información contenida en ellos. 

Ignora todas las palabras que aparecen en menos de X = 30 correos electrónicos del conjunto de 4000 mensajes de correo electrónico, esto es tanto un medio de prevenir el ajuste excesivo y de mejorar la escalabilidad. Para cada correo electrónico, transformarlo en un vector de características $\mathbf{x}$ donde la i-ésima entrada, $x_i$, es 1 si la i-ésima palabra del vocabulario aparece en el correo electrónico y 0 en caso contrario.

In [4]:
#Separamos las palabras y calculamos su frecuencia para todos los correos
from collections import Counter
word_freq = Counter(" ".join(X_train.Text.str.lower().values.tolist()).split(" ")).items()

#type(word_freq)
word_freq = list(word_freq)

# Colocamos las frecuencias en tablas y restringimos para frecuencias mayores a 30
wf = pd.DataFrame(word_freq,columns=['Word','Freq'])
wf = wf[wf['Freq']>=30]

print('Numero de palabras más frecuentes:',len(wf))
words = wf.Word
wf.head()

Numero de palabras más frecuentes: 3251


,Word,Freq
0,todai,805
1,s,8157
2,headlin,64
3,from,5851
4,the,44774


In [5]:
def validar_palabras(X,words):
    # n filas = Nº oraciones, m columnas = Nºpalabras
    n = X.shape[0]
    m = words.shape[0]

    # Declaramos una matriz donde guardaremos las evaluaciones de las palabras
    values = np.zeros((n,m), dtype='int32')
    k = 0
    for word in words:
        for i in range(n):
            if word in X.Text.iloc[i]:
                values[i,k]=1
        k = k+1
    index = X.index.values # indices de oraciones
    words_in = pd.DataFrame(values,columns = words, index = index) # Cambio a DataFrame
    X = pd.concat((X,words_in),axis=1)
    return X

In [6]:
X_train = validar_palabras(X_train,words)
X_train[words].head()

,todai,s,headlin,from,the,regist,to,unsubscrib,thi,daili,...,dollarb,lindowso,dollarftoc,apg,curlin,nri,dollarac,numbercnumberabafnumberefnumbercnumb,dqogicagicanumberyni,snore
1305,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2525,0,1,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3966,0,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3394,0,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2961,0,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


**3** .Implementar las funciones **entrenamiento_perceptron(datos)** y **prueba_perceptron(w, datos)**.

La función `entrenamiento_perceptron(datos)` entrena un clasificador perceptron usando los ejemplos proporcionados a la función, y devuelve `w`, `k`, `iter`, el vector de clasificación final, el número de actualizaciones (errores) realizados y el número de pasadas los datos, respectivamente. 

Se puede  suponer que los datos de entrada proporcionados a la función son linealmente separables (por lo que el criterio de parada debe ser que todos los puntos estén clasificados correctamente).

Para el caso  `w·x = 0`, predecir la clase +1 (spam). Para este ejercicio, no es necesario agregar una función de sesgo al vector de características 

La función `prueba_perceptron(w, datos)` debe tomar como entrada el vector de peso `w`  (el vector de clasificación a utilizar) y un conjunto de ejemplos. La función debe devolver el error de prueba, es decir, la fracción de ejemplos que son mal clasificados por w.

In [7]:
def activacion(x):
    for i in range(len(x)):
        x[i] = 1 if x[i] > 0 else 0
    return x

In [8]:
def entrenamiento_perceptron(x ,yt): # x dataframe con caracteristicas, yt series a predecir
    yt = yt.values.astype(np.float64)
    w = np.random.rand(x.shape[1]+1) # vector de pesos aleatorios 
    tasa = 0.05
    iters = 0
    k = 0
    sqr_delta = 1.0
    n = x.shape[0]
    while sqr_delta > 0 and iters <1000:
        for i in range(n):
            val = x.iloc[i].dot(w[1:])+w[0]
            if val > 0 and not yt[i]:
                k += 1
                w[1:] -= tasa * x.iloc[i]
                w[0] -= tasa
            elif val <=0 and yt[i]:
                w[1:] += tasa * x.iloc[i]
                w[0] += tasa 
                k += 1
        yp = activacion(x.dot(w[1:]).values + w[0]*1) # evalua toda la data de entrenamiento
        sqr_delta = (yt-yp).dot(yt-yp) # suma de cuadrados de la diferencia
        iters +=1
    return w,k,iters

In [9]:
def prueba_perceptron(w,xp,yp):
    type(w)
    error = 0
    n = xp.shape[0]
    yp = yp.values.astype(np.float64)
    for i in range(n):
        val = xp.iloc[i].dot(w[1:])
        val = 1 if val>0 else 0 # val representa f(x_i*w_i)
        if yp[i] != val:
            error +=1
    return 1.0*error/n

**4** .Entrene el clasificador lineal usando su conjunto de entrenamiento. ¿Cuántos errores se cometen antes de que termine el algoritmo? Prueba tu implementación `prueba_perceptron` con  los parámetros aprendidos y los datos de entrenamiento, asegurándose de que el error de entrenamiento es cero. A continuación, clasifique los correos electrónicos en su conjunto de validación. ¿Qué es el error de validación?

In [10]:
w,errores,iters = entrenamiento_perceptron(X_train[words] ,y_train)
print('Cantidad de Errores:',errores)
error_ent = prueba_perceptron(w ,X_train[words],y_train)
print('Error de entrenamiento:',error_ent)

Cantidad de Errores: 705
Error de entrenamiento: 0.0


In [12]:
# Verificamos si el conjunto de validación contiene las palabras de la tabla de frecuencias
X_test = validar_palabras(X_test,words)
X_test[words].head()

,todai,s,headlin,from,the,regist,to,unsubscrib,thi,daili,...,dollarb,lindowso,dollarftoc,apg,curlin,nri,dollarac,numbercnumberabafnumberefnumbercnumb,dqogicagicanumberyni,snore
1367,0,1,0,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3277,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2449,1,1,0,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
198,0,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
280,1,1,0,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Clasificamos para ver el error
error_val = prueba_perceptron(w,X_test[words],y_test)
print('Error de validacion:',error_val)

Error de validacion: 0.021


**5** .Para entender mejor cómo funciona este clasificador de spam, podemos inspeccionar los parámetros para ver qué palabras cree que el clasificador es la más predictiva a spam. Usando la lista de vocabulario junto con los parámetros aprendidos en la pregunta anterior, muestra las 15 palabras con los pesos más positivos. ¿Cuáles son las  15 palabras que tienen los pesos más negativos?.


In [14]:
# Obtenemos los indices de los pesos ordenados ascendente y descendentemente
asc = np.argsort(w)
desc = asc[::-1] #invertimos el orden
print('Palabras con peso más positivo')
for i in range(15):
    print(words.iloc[desc[i]],w[desc[i]])
print('\nPalabras con peso más negativo')
for i in range(15):
    print(words.iloc[asc[i]],w[asc[i]])

Palabras con peso más positivo
here 3.48663144223
modul 2.89711320151
increas 1.96451409475
cf 1.96045358656
it 1.70074846712
transfer 1.62692987028
absolut 1.6118734619
iso 1.6053748285
regul 1.59958980253
percent 1.59305421279
less 1.57111245079
mount 1.56858036372
agreement 1.49666605962
sight 1.46976529788
formal 1.43944447946

Palabras con peso más negativo
assum -2.16000615761
inlin -1.68974677994
crash -1.62672963381
sent -1.59725957667
club -1.5843452379
member -1.57405613873
numbertnumb -1.56114885557
tel -1.5489273553
envelop -1.54726047621
non -1.45807757648
regist -1.4330493413
protest -1.43260445784
plc -1.43024666257
mai -1.41417988182
music -1.40634296193


**6** . Implementar el algoritmo de perceptron promedio, que es el mismo que tu implementación actual, pero que, en lugar de devolver el vector de peso final, devuelve el promedio de todos los vectores de peso considerados durante el algoritmo (incluyendo ejemplos donde no se cometió ningún error). El promedio reduce la varianza entre los diferentes vectores, y es un poderoso medio para evitar que el algoritmo de aprendizaje sea sobrefijado (sirviendo como un tipo de regularización).

In [16]:
def entrenamiento_perceptron_prom(x ,yt): # x dataframe con caracteristicas, yt series a predecir
    yt = yt.values.astype(np.float64)
    w = np.random.rand(x.shape[1]+1) # vector de pesos aleatorios 
    tasa = 0.05
    iters = 0
    k = 0
    sqr_delta = 1.0
    n = x.shape[0]
    w_mean = w/n
    while sqr_delta > 0  and iters <1000:
        for i in range(n):
            val = x.iloc[i].dot(w[1:])+w[0]
            if val > 0 and not yt[i]:
                k += 1
                w[1:] -= tasa * x.iloc[i]
                w[0] -= tasa
            elif val <=0 and yt[i]:
                w[1:] += tasa * x.iloc[i]
                w[0] += tasa 
                k += 1
            w_mean += w/n 
        iters +=1
        # Predice con la data de entrenamiento
        #yp = activacion(x.dot(w_mean[1:]/iters).values + w_mean[0]/iters*1)  # Considera pesos promedio 
        yp = activacion(x.dot(w[1:]).values + w[0]*1)
        sqr_delta = (yt-yp).dot(yt-yp) # suma de cuadrados de la diferencia de prediccion vs esperado
    w_mean = w_mean/iters
    return w_mean,k,iters

**7** .Uno debe esperar que el error de prueba disminuye a medida que aumenta la cantidad de datos de entrenamiento. Usando sólo las primeras N filas de tus datos de entrenamiento, ejecute tanto el algoritmo del perceptron como el  algoritmo de perceptron promedio en este conjunto de entrenamiento  y evalúe el error de validación correspondiente (usando todos los datos de validación). Hacer esto para N = 100, 200, 400, 800, 2000, 4000 y crear un gráfico del error de validación de ambos algoritmos como una función de N.

In [24]:
# Tu respuesta

**8** . También para N = 100, 200, 400, 800, 2000, 4000, crea un diagrama del número de iteraciones para el perceptrón como una función de N, donde por iteración nos referimos a un paso completo a través de los datos de entrenamiento. A medida que aumenta la cantidad de datos de entrenamiento, el margen del conjunto de entrenamiento disminuye, lo que generalmente conduce a un aumento en el número de iteraciones que el perceptron toma para converger.


In [ ]:
# Tu respuesta

**9** . Prueba varias configuraciones de los algoritmos usando todos los 4000 datos de entrenamiento y encuentra una buena configuración con un error pequeño en su conjunto de validación. En particular, trata de cambiar la elección del algoritmo perceptron y el número máximo de iteraciones. 

Reporta el error de validación para varias de las configuraciones, qué configuración funciona mejor?

Combina el conjunto de entrenamiento y el conjunto de validación (es decir, utiliza `entrenamiento_spam.txt`) y aprenda usando la mejor de las configuraciones encontradas anteriormente.

¿Cuál es el error en el conjunto de pruebas, es decir, en  `prueba_spam.txt`?

In [ ]:
# Tu respuesta